In [1]:
web = [[0,0],[0,1],[1,0],[1,2]]
n = 3
# get out-links
outLinks = {}
for each in web:
    if each[0] in outLinks:
        outLinks[each[0]] += 1
    else:
        outLinks[each[0]] = 1

In [2]:
# form column adjacency matrix
import numpy as np

M = np.zeros((n,n))
for each in web:
    i = each[0]
    j = each[1]
    M[j][i] = 1. / outLinks[i]

print 'Dense matrix =' 
print M

Dense matrix =
[[ 0.5  0.5  0. ]
 [ 0.5  0.   0. ]
 [ 0.   0.5  0. ]]


In [5]:
# transform M into a sparse matrix
from scipy import *
from scipy.sparse import *

S = csr_matrix(M)
print 'Sparse matrix ='
print S 

Sparse matrix =
  (0, 0)	0.5
  (0, 1)	0.5
  (1, 0)	0.5
  (2, 1)	0.5


In [6]:
# create rank vector r (r sums to 1)
r = np.array([1./n] * n)

# compute page rank
epsilon = 1.0e-8
delta = 1
iter = 0
while delta > epsilon:
    rNew = S.dot(r)
    delta = sum(map(abs,rNew - r))
    r = rNew
    iter += 1
    
print 'PageRank =',r
print 'Iteration =', iter

PageRank = [  2.08940652e-08   1.29132425e-08   7.98082275e-09]
Iteration = 79


In [7]:
def computePageRank(M, r, epsilon=1.0e-8):
    """
    Function to compute page rank with a given adjacency matrix and rank vector
    """
    delta = 1
    iter = 0
    while delta > epsilon:
        rNew = M.dot(r)
        delta = sum(map(abs,rNew - r))
        r = rNew
        iter += 1
    print 'PageRank =',r
    print 'Iteration =', iter

    return r

In [8]:
# construct a stochastic matrix (columns sum to 1) to account for dead end

MNew = np.copy(M)
for i in range(3):
    MNew[i][2] += 1./3
print 'New M matrix:'
print MNew
print

# compute page rank
r = np.array([1./n] * n)
computePageRank(MNew, r)

New M matrix:
[[ 0.5         0.5         0.33333333]
 [ 0.5         0.          0.33333333]
 [ 0.          0.5         0.33333333]]

PageRank = [ 0.46153846  0.30769231  0.23076923]
Iteration = 19


array([ 0.46153846,  0.30769231,  0.23076923])

In [9]:
# use google matrix and teleport parameter beta
beta = 0.8
e = ones((n,n))

A = M.dot(beta) + (1-beta) * 1./n * e
print 'My google matrix:'
print A
print 

r = np.array([1./n] * n)
res = computePageRank(A, r)

print
print 'PageRank * 1.0e8 =',res*1.0e8

My google matrix:
[[ 0.46666667  0.46666667  0.06666667]
 [ 0.46666667  0.06666667  0.06666667]
 [ 0.06666667  0.46666667  0.06666667]]

PageRank = [  1.76227715e-08   1.18132599e-08   8.95626491e-09]
Iteration = 82

PageRank * 1.0e8 = [ 1.76227715  1.18132599  0.89562649]


In [10]:
# use sparse matrix with teleport parameter

r = np.array([1./n] * n)
epsilon = 1.0e-8
delta = 1
one = ones(n)
while delta > epsilon:
    rNew = S.dot(beta).dot(r) + (1-beta) / n * one
    delta = sum(map(abs,rNew - r))
    r = rNew

print 'PageRank =' ,r

PageRank = [ 0.21212122  0.15151516  0.12727273]


In [14]:
def pageRank(web, n, beta=0.8, epsilon = 1.0e-8):
    """
    Compute the pagerank of n nodes in a given directed web graph
    """
    # get out-links
    outLinks = {}
    for each in web:
        if each[0] in outLinks:
            outLinks[each[0]] += 1
        else:
            outLinks[each[0]] = 1

    # form base matrix 
    M = np.zeros((n,n))
    for each in web:
        i = each[0]
        j = each[1]
        M[j][i] = 1. / outLinks[i]
    
    # form sparse matrix
    S = csr_matrix(M)
    
    # create rank vector r (r sums to 1)
    r = np.array([1./n] * n)

    # compute page rank     
    delta = 1
    one = ones(n)
    while delta > epsilon:
        rNew = S.dot(beta).dot(r) + (1-beta) / n * one
        delta = sum(map(abs,rNew - r))
        r = rNew

    return r


In [15]:
# test scalable page rank algorithm on small case

pageRank(web, 3)

array([ 0.21212122,  0.15151516,  0.12727273])

In [16]:
# deploy on large graph

# read the file

myFile = open('web-Google.txt', 'r')

# skip the first few description lines
for _ in range(4):
    myFile.readline()

# start reading each line
largeWeb = []
t = myFile.readline()
while t:
    largeWeb.append([int(x) for x in (t.split())]) # strip off unrelated characters such as '\t', '\r', etc.
    t = myFile.readline()
    
# verify I have read all lines properly
print largeWeb[0], largeWeb[-1]

[0, 11342] [916425, 837379]


In [17]:
# find n
n = 0
for i in range(len(largeWeb)):
    current = max(largeWeb[i][0], largeWeb[i][1])
    if current > n:
        n = current
# number of nodes is the maximum number plus 1, accounting for node 0
n += 1
print n

916428


In [ ]:
# compute page rank for large web graph
rank = pageRank(largeWeb, n)

In [ ]:
print rank[99]